In [1]:
!pip install -U google-cloud_aiplatform langchain langchain-google-genai langgraph langchain_experimental langchain_google_vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7

In [44]:
from datetime import datetime
from typing import Optional, Sequence, TypedDict, Dict, List, Union, Any
from typing_extensions import Annotated

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

from keras.models import load_model
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    AIMessage,
    SystemMessage,
)
from langchain_core.language_models import BaseChatModel, LLM
from langchain_core.outputs import Generation, ChatResult, ChatGeneration
from langchain_core.runnables import Runnable
from langchain_core.tools import Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_tool

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import (
    AgentState,
    add_messages,
    IsLastStep,
    RemainingSteps,
)

from langchain_google_vertexai import ChatVertexAI

from vertexai import init as vertexai_init
import holidays


In [45]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/GEMINI_JSON_ACTIVE.json"

In [46]:
import os
from google.oauth2 import service_account
from langchain_google_vertexai.chat_models import ChatVertexAI
from langchain_core.messages import HumanMessage

# Load credentials explicitly from service account file
credentials = service_account.Credentials.from_service_account_file(
    "/content/GEMINI_JSON_ACTIVE.json"
)

# Initialize ChatVertexAI with explicit credentials
llm = ChatVertexAI(
    model="gemini-2.5-pro",
    project="steady-bonsai-467007-g0",
    location="us-central1",
    max_output_tokens=1024,
    temperature=0.7,
    credentials=credentials  # ✅ This overrides the default fallback
)

llm.invoke("HI")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 1, 'candidates_token_count': 9, 'total_token_count': 610, 'prompt_tokens_details': [{'modality': 1, 'token_count': 1}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 9}], 'thoughts_token_count': 600, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -9.127059088812935, 'model_name': 'gemini-2.5-pro'}, id='run--3f779aa9-75f5-468f-907f-0a8b198ebc5c-0', usage_metadata={'input_tokens': 1, 'output_tokens': 9, 'total_tokens': 610, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 600}})

In [47]:
#Corecast

In [83]:
df = None
df2 = None
coef_global = []
XGB_MAPE = None
LSTM_MAPE = None
forecast_df = None
@tool
def preprocess(input: str) -> str:
    """
    Preprocess the call volume dataset for modeling.

    Steps performed:
    1. Loads and aggregates Costco call volume data.
    2. Extracts time-based features: day of week, month, quarter, holidays, etc.
    3. Performs sinusoidal encoding for cyclic features.
    4. Engineers MOB (Months on Book) features using new account data.
    5. Merges all engineered features into a clean DataFrame (global `df`).

    Returns:
        str: Success message or error encountered during processing.
    """

    global df
    try:
        df = pd.read_csv("/content/Costco_interval_data_lstm.csv")
        df = df.groupby("Date").agg({"Call Volume": "sum", "AHT": "mean"}).reset_index()
        df.set_index("Date", inplace=True)
        df.sort_index(inplace=True)
        df.drop(columns="AHT", inplace=True)
    except Exception as e:
        return f"Error in reading and preprocessing df: {str(e)}"

    try:
        df.index = pd.to_datetime(df.index, format='mixed')
        df["day_of_week"] = df.index.dayofweek
        df["month"] = df.index.month
        df["quarter"] = df.index.quarter
        df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)
        df["day_of_week_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
        df["day_of_week_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)
        df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
        df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
        df["week_of_month"] = (df.index.day - 1) // 7 + 1

        # Add holiday columns
        major_holidays = pd.to_datetime([
            "2018-01-01", "2018-07-04", "2018-11-22", "2018-12-25",
            "2019-01-01", "2019-07-04", "2019-11-28", "2019-12-25",
            "2020-01-01", "2020-07-04", "2020-11-26", "2020-12-25",
            "2021-01-01", "2021-07-04", "2021-11-25", "2021-12-25",
            "2022-01-01", "2022-07-04", "2022-11-24", "2022-12-25",
            "2023-01-01", "2023-07-04", "2023-11-23", "2023-12-25",
            "2024-01-01", "2024-07-04", "2024-11-28", "2024-12-25",
            "2025-01-01", "2025-07-04", "2025-11-27", "2025-12-25"
        ])
        minor_holidays = pd.to_datetime([
            "2018-02-19", "2018-10-08", "2018-12-24", "2018-12-31",
            "2019-02-18", "2019-10-14", "2019-12-24", "2019-12-31",
            "2020-02-17", "2020-10-12", "2020-12-24", "2020-12-31",
            "2021-02-15", "2021-10-11", "2021-12-24", "2021-12-31",
            "2022-02-21", "2022-10-10", "2022-12-24", "2022-12-31",
            "2023-02-20", "2023-10-09", "2023-12-24", "2023-12-31",
            "2024-02-19", "2024-10-14", "2024-12-24", "2024-12-31",
            "2025-02-17", "2025-10-13", "2025-12-24", "2025-12-31"
        ])
        positive_holidays = pd.to_datetime([
            "2018-11-23", "2018-11-26", "2019-11-29", "2019-12-02",
            "2020-11-27", "2020-11-30", "2021-11-26", "2021-11-29",
            "2022-11-25", "2022-11-28", "2023-11-24", "2023-11-27",
            "2024-11-29", "2024-12-02", "2025-11-28", "2025-12-01"
        ])
        df["is_major_holiday"] = df.index.isin(major_holidays).astype(int)
        df["is_minor_holiday"] = df.index.isin(minor_holidays).astype(int)
        df["is_positive_holiday"] = df.index.isin(positive_holidays).astype(int)

        # MOB Feature Engineering
        df["Year"] = df.index.year
        df["Month"] = df.index.month

        new_accounts_df = pd.read_csv(r"/content/Costco_new_accounts.csv", parse_dates=["Date"])
        mob_cols = [f"MOB_{i}" for i in range(24)] + ["MOB_>24"]
        result_df = pd.DataFrame(columns=["Date"] + mob_cols)
        result_df["Date"] = new_accounts_df["Date"]

        for i in range(len(new_accounts_df)):
            values = [0] * len(mob_cols)
            values[0] = new_accounts_df.loc[i, "New_Accounts"]
            for j in range(1, 24):
                if i - j >= 0:
                    values[j] = result_df.loc[i - 1, f"MOB_{j - 1}"]
            if i - 23 >= 0:
                values[-1] = result_df.loc[i - 1, "MOB_>24"] + result_df.loc[i - 1, "MOB_23"]
            result_df.loc[i, mob_cols] = values

        result_df.index = pd.to_datetime(result_df["Date"])
        result_df.drop(columns="Date", inplace=True)
        result_df["Year"] = result_df.index.year
        result_df["Month"] = result_df.index.month

        df = df.merge(result_df, on=["Year", "Month"], how="left").set_index(df.index)
        df.drop(columns=["Year", "Month"], inplace=True)
    except Exception as e:
        return f"Error in feature engineering: {str(e)}"

    return "✅ Preprocessing completed successfully."

@tool
def MAPE_Comparision_tool(input: str) -> str:
    """
    Compares the MAPE (Mean Absolute Percentage Error) of LSTM and XGBoost forecasts
    over the last 14 days of actual data.

    Steps performed:
    1. Loads actual vs predicted values from Excel.
    2. Calculates total error percentage for LSTM and XGBoost forecasts.
    3. Sets global variables `LSTM_MAPE` and `XGB_MAPE`.
    4. Selects the model with lower MAPE.

    Returns:
        str: Either 'forecast_lstm_tool' or 'forecast_xgb_tool' based on comparison.
    """

    global df2, LSTM_MAPE, XGB_MAPE
    try:
        df2 = pd.read_excel("/content/ACTUAL VS PRED XGB AND LSTM.xlsx")
        LSTM_MAPE = (abs(df2['Call Volume Actual'].sum() - df2['Call Volume LSTM Forecast'].sum() ) / df2['Call Volume Actual'].sum() ) * 100
        XGB_MAPE = (abs(df2['Call Volume Actual'].sum()  - df2['Call Volume XGB Forecast'].sum() ) / df2['Call Volume Actual'].sum() ) * 100
        print(f"mape_lstm is {LSTM_MAPE} and mape_xgb is {XGB_MAPE}")

        return "forecast_lstm_tool" if LSTM_MAPE <= XGB_MAPE else "forecast_xgb_tool"
    except Exception as e:
        return f"Error in MAPE Comparison tool: {str(e)}"

@tool
def forecast_lstm_tool(input: str) -> str:
    """
    Generates 180-day call volume forecasts using a pretrained LSTM model.

    Steps performed:
    1. Loads the trained LSTM model.
    2. Scales historical data using MinMaxScaler.
    3. Extracts the last 540 time steps as input sequence.
    4. Predicts scaled values and inverses only the call volume.
    5. Builds a forecast DataFrame with future dates and predicted values.

    Returns:
        str: Forecasted values as a formatted string DataFrame.
    """

    global df,forecast_df
    model = load_model("/content/Costco_LSTM_model_v5.keras")

    feature_cols = [
        'Call Volume', 'day_of_week', 'month', 'quarter', 'is_weekend',
        'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
        'week_of_month', 'is_major_holiday', 'is_minor_holiday', 'is_positive_holiday',
        'MOB_0', 'MOB_1', 'MOB_2', 'MOB_3', 'MOB_4', 'MOB_5', 'MOB_6', 'MOB_7',
        'MOB_8', 'MOB_9', 'MOB_10', 'MOB_11', 'MOB_12', 'MOB_13', 'MOB_14', 'MOB_15',
        'MOB_16', 'MOB_17', 'MOB_18', 'MOB_19', 'MOB_>24','MOB_>24'
    ]

    data = df[feature_cols].copy()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    SEQ_LEN = 540
    last_seq = scaled_data[-SEQ_LEN:, :]
    input_seq = last_seq.reshape(1, SEQ_LEN, len(feature_cols))

    pred_scaled = model.predict(input_seq, verbose=0).flatten()

    call_vol_index = feature_cols.index('Call Volume')
    call_volume_scaler = MinMaxScaler()
    call_volume_scaler.min_ = np.array([scaler.min_[call_vol_index]])
    call_volume_scaler.scale_ = np.array([scaler.scale_[call_vol_index]])

    forecast_unscaled = call_volume_scaler.inverse_transform(pred_scaled.reshape(-1, 1))

    last_date = df.index[-1]
    forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=180)

    forecast_df = pd.DataFrame({
        "Date": forecast_dates,
        "Predicted_Call_Volume": forecast_unscaled.flatten()
    })

    return forecast_df.to_string(index=False)

@tool
def forecast_xgb_tool(input: str) -> str:
    """
    Generates 180-day call volume forecasts using a pretrained XGB model.

    Steps performed:
    1. Loads the trained XGB model.
    2. Scales historical data using MinMaxScaler.
    3. Extracts the last 540 time steps as input sequence.
    4. Predicts scaled values and inverses only the call volume.
    5. Builds a forecast DataFrame with future dates and predicted values.

    Returns:
        str: Forecasted values as a formatted string DataFrame.
    """

    global df,forecast_df
    model = load_model("/content/Costco_LSTM_model_v5.keras")

    feature_cols = [
        'Call Volume', 'day_of_week', 'month', 'quarter', 'is_weekend',
        'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
        'week_of_month', 'is_major_holiday', 'is_minor_holiday', 'is_positive_holiday',
        'MOB_0', 'MOB_1', 'MOB_2', 'MOB_3', 'MOB_4', 'MOB_5', 'MOB_6', 'MOB_7',
        'MOB_8', 'MOB_9', 'MOB_10', 'MOB_11', 'MOB_12', 'MOB_13', 'MOB_14', 'MOB_15',
        'MOB_16', 'MOB_17', 'MOB_18', 'MOB_19', 'MOB_>24','MOB_>24'
    ]

    data = df[feature_cols].copy()
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    SEQ_LEN = 540
    last_seq = scaled_data[-SEQ_LEN:, :]
    input_seq = last_seq.reshape(1, SEQ_LEN, len(feature_cols))

    pred_scaled = model.predict(input_seq, verbose=0).flatten()

    call_vol_index = feature_cols.index('Call Volume')
    call_volume_scaler = MinMaxScaler()
    call_volume_scaler.min_ = np.array([scaler.min_[call_vol_index]])
    call_volume_scaler.scale_ = np.array([scaler.scale_[call_vol_index]])

    forecast_unscaled = call_volume_scaler.inverse_transform(pred_scaled.reshape(-1, 1))

    last_date = df.index[-1]
    forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=180)

    forecast_df = pd.DataFrame({
        "Date": forecast_dates,
        "Predicted_Call_Volume": forecast_unscaled.flatten()
    })

    return forecast_df.to_string(index=False)




def linear_reg(state:AgentState):
    """
    Run linear regression on the data
    """
    global df, coef_global
    X = df.drop(columns=["Call Volume"])
    y = df["Call Volume"]
    model = LinearRegression()
    X.fillna(0,inplace=True)
    model.fit(X, y)
    coeffs = pd.Series(model.coef_, index=X.columns).sort_values(key=abs, ascending=False)
    coef_global.append(coeffs)
    return coeffs


def conditional_edge(state:AgentState):
  if min(XGB_MAPE,LSTM_MAPE)<=5:
    return "END"
  else:
      human_input = str(input("PRESS 'Y' to Run co-eff analysis or 'N' to END"))
      if human_input=='Y':
        return "linear_reg"
      else:
        return "END"

tools = [preprocess, forecast_lstm_tool,forecast_xgb_tool,MAPE_Comparision_tool]
llm = ChatVertexAI(
    model="gemini-2.5-flash-lite",
    project="steady-bonsai-467007-g0",
    location="us-central1",
    max_output_tokens=1024,
    temperature=0.7,
    credentials=credentials  # ✅ This overrides the default fallback
)
bound_llm = llm.bind_tools(tools)

agent_node = create_react_agent(bound_llm, tools=tools, state_schema=AgentState)
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.add_node("linear_reg", linear_reg)
graph.set_entry_point("agent")
graph.add_conditional_edges("agent",conditional_edge,{"END":END, "linear_reg":"linear_reg"})

runnable = graph.compile()
prompt = "Preprocess the dataframe 'df', compute the MAPE value for the last 14 days, and based on that, run the preprocessed data through either an LSTM or XGBoost model."
output = runnable.invoke({"messages": [{"role": "user", "content": prompt}]})

print("\nFinal output from agent:\n")
for msg in output["messages"]:
    if hasattr(msg, "name"):
        print(f"[TOOL: {msg.name}] {msg.content}")

/tmp/ipython-input-83-3239179000.py:80: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_accounts_df = pd.read_csv(r"/content/Costco_new_accounts.csv", parse_dates=["Date"])


mape_lstm is 8.431706098003085 and mape_xgb is 6.0297874805059966


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


PRESS 'Y' to Run co-eff analysis or 'N' to ENDN

Final output from agent:

[TOOL: None] Preprocess the dataframe 'df', compute the MAPE value for the last 14 days, and based on that, run the preprocessed data through either an LSTM or XGBoost model.
[TOOL: None] 
[TOOL: preprocess] ✅ Preprocessing completed successfully.
[TOOL: None] 
[TOOL: MAPE_Comparision_tool] forecast_xgb_tool
[TOOL: None] 
[TOOL: forecast_xgb_tool]       Date  Predicted_Call_Volume
2025-01-01           36482.796875
2025-01-02           35639.609375
2025-01-03           35240.878906
2025-01-04           17417.574219
2025-01-05           15371.204102
2025-01-06           35267.050781
2025-01-07           35814.933594
2025-01-08           36559.531250
2025-01-09           36100.503906
2025-01-10           35635.503906
2025-01-11           17405.591797
2025-01-12           16260.489258
2025-01-13           35028.109375
2025-01-14           36643.683594
2025-01-15           36600.578125
2025-01-16           36022.8476

In [84]:
# df_holidays = None
# holiday_forecast = None
@tool
def generate_holidays_dataframe(input: str) -> str:
    """
    Generate a DataFrame of U.S. federal holidays from 2020 to 2025.
    Returns:
        dict: {"df_holidays": pd.DataFrame}
    """
    global df_holidays

    us_holidays = holidays.US(years=range(2020, 2026))
    holiday_df = pd.DataFrame(us_holidays.items(), columns=['Date', 'Holiday'])
    df_holidays = holiday_df.sort_values('Date').reset_index(drop=True)
    return df_holidays


@tool
def identify_holidays(input: str) -> str:
    """
    Identify holiday dates in the forecast dataset and compare forecasted call volumes with the average actuals
    for the same weekday over the past 3 weeks to estimate the forecasted holiday impact.

    Returns:
        pd.DataFrame: A DataFrame showing:
            - Holiday date
            - Holiday name
            - Day of the week
            - 3-week average actuals for the same weekday
            - Forecasted volume
            - Percentage difference indicating the forecasted impact
    """
    global df, forecast_df, df_holidays,holiday_forecast
    df = df.reset_index()

    forecast_df['Date'] = pd.to_datetime(forecast_df['Date'])

    df['Date'] = pd.to_datetime(df['Date'])
    df_holidays['Date'] = pd.to_datetime(df_holidays['Date'])



    forecast_df['day_of_week'] = forecast_df['Date'].dt.day_name()
    df['day_of_week'] = df['Date'].dt.day_name()
    df_holidays['day_of_week'] = df_holidays['Date'].dt.day_name()

    # Join forecasted holidays

    holiday_forecast = forecast_df.merge(df_holidays[['Date', 'Holiday']], on="Date", how="inner")
    holiday_forecast = holiday_forecast.merge(df[['Date', 'Call Volume']], on="Date", how="inner")

    holiday_forecast['Day of Week'] = holiday_forecast['Date'].dt.day_name()

    # Compute 3-week average call volume for the same weekday

    dow_averages = []
    for date, dow in zip(holiday_forecast['Date'], holiday_forecast['Day of Week']):
        start_date = date - pd.Timedelta(weeks=3)
        filtered = df[(df['Date'] < date) & (df['Date'] >= start_date)]
        avg = filtered[filtered['day_of_week'] == dow]['Call Volume'].mean()
        dow_averages.append(avg)

    holiday_forecast['dow_avg_3w'] = dow_averages

    holiday_forecast['Impact_Captured'] = round(
        (holiday_forecast['Predicted_Call_Volume'] - holiday_forecast['dow_avg_3w']) /
        holiday_forecast['dow_avg_3w'] * 100, 2
    )

    return holiday_forecast[['Date', 'Holiday', 'Day of Week', 'dow_avg_3w', 'Predicted_Call_Volume', 'Impact_Captured']]


@tool
def holiday_impact(input: str) -> str:
    """
    Evaluate the *actual* impact of holidays on call volume by comparing actuals to a 3-week weekday average.

    Returns:
        pd.DataFrame: A summarized DataFrame grouped by holiday, showing:
            - Holiday name
            - Mean actual call volume on the holiday
            - Mean percentage difference compared to normal same-day-of-week volumes (Actual Impact)
    """
    global holiday_forecast, df_holidays, result

    holiday_forecast['Date'] = pd.to_datetime(holiday_forecast['Date'])
    df_holidays['Date'] = pd.to_datetime(df_holidays['Date'])

    # Ensure there’s only one 'Holiday' column
    if 'Holiday' not in holiday_forecast.columns:
        holiday_forecast = pd.merge(holiday_forecast, df_holidays[['Date', 'Holiday']], on='Date', how='left')

    merged_df = holiday_forecast.copy()
    merged_df['Holiday'] = merged_df['Holiday'].fillna('Normal Day')
    merged_df['day_of_week'] = merged_df['Date'].dt.day_name()

    dow_avg_3w = []

    for idx, row in merged_df.iterrows():
        current_date = row['Date']
        current_dow = row['day_of_week']

        # Get previous 3 weeks of same-day-of-week before the current date
        start_date = current_date - pd.Timedelta(weeks=3)
        mask = (holiday_forecast['Date'] < current_date) & (holiday_forecast['Date'] >= start_date)
        previous_3w = holiday_forecast[mask].copy()
        previous_3w['day_of_week'] = previous_3w['Date'].dt.day_name()
        avg = previous_3w[previous_3w['day_of_week'] == current_dow]['Call Volume'].mean()
        dow_avg_3w.append(avg)

    merged_df['dow_avg_3w'] = dow_avg_3w

    merged_df['Actual_Impact'] = round(
        (merged_df['Call Volume'] - merged_df['dow_avg_3w']) /
        merged_df['dow_avg_3w'] * 100, 2
    )

    result = merged_df.groupby("Holiday", as_index=False).agg({
        "Call Volume": "mean",
        "Actual_Impact": "mean"
    })

    return result


tools = [generate_holidays_dataframe, identify_holidays,holiday_impact]

llm = ChatVertexAI(
    model="gemini-2.5-pro",
    project="steady-bonsai-467007-g0",
    location="us-central1",
    max_output_tokens=1024,
    temperature=0,
    credentials=credentials
)
bound_llm = llm.bind_tools(tools)

agent_node = create_react_agent(bound_llm, tools=tools, state_schema=AgentState)
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")


runnable = graph.compile()
runnable.invoke({
    "messages": [{"role": "user", "content": "Generate a DataFrame of U.S. federal holidays from 2020 to 2025"}]
})

runnable.invoke({
    "messages": [{"role": "user", "content": "Identify holiday dates in the forecast dataset and compare forecasted call volumes with the average actuals using 'forecast_df' and 'df' "}]
})

output=runnable.invoke({
    "messages": [{"role": "user", "content": "Evaluate the *actual* impact of holidays on call volume by comparing actuals to a 3-week weekday average. with 'holiday_forecast' and 'df_holidays' "}]
})

print("\nFinal output from agent:\n")
for msg in output["messages"]:
    if hasattr(msg, "name"):
        print(f"[TOOL: {msg.name}] {msg.content}")


Final output from agent:

[TOOL: None] Evaluate the *actual* impact of holidays on call volume by comparing actuals to a 3-week weekday average. with 'holiday_forecast' and 'df_holidays' 
[TOOL: None] 
[TOOL: holiday_impact]                       Holiday  Call Volume  Actual_Impact
0  Martin Luther King Jr. Day      37183.0            NaN
1              New Year's Day      11318.0            NaN
2       Washington's Birthday      37015.0            NaN
[TOOL: None] The actual impact of holidays on call volume is as follows: 

| Holiday | Call Volume | Actual_Impact |
| :--- | :--- | :--- |
| Martin Luther King Jr. Day | 37183.0 | NaN |
| New Year's Day | 11318.0 | NaN |
| Washington's Birthday | 37015.0 | NaN |

This table displays the mean actual call volume on each holiday, but the actual impact, measured as the percentage difference compared to normal same-day-of-week volumes, could not be determined.


In [85]:
result.head()

,Holiday,Call Volume,Actual_Impact
0,Martin Luther King Jr. Day,37183.0,NaN
1,New Year's Day,11318.0,NaN
2,Washington's Birthday,37015.0,NaN


In [54]:
#SesoCast

In [88]:
df_history = df.iloc[-28:]
df_history_2 = df.iloc[-28:]
historical_periods = None
historical_periods_fcst = None

@tool
def get_historical_periods_dynamic_actual(input: str) -> str:
    """
    Extracts the same date range in previous years from a DataFrame, dynamically determining available years.
    Returns:
        dict: A dictionary where keys are years and values are DataFrames
              containing the corresponding period. Returns an empty dictionary if there are issues.
    """
    # Ensure date column is datetime
    global df,df_history,historical_periods

    start_date = None
    end_date = None

    date_column='Date'
    df_history = df_history.reset_index()
    df_history[date_column] = pd.to_datetime(df_history[date_column])

    # Determine start and end dates if not provided
    if start_date is None or end_date is None:
        end_date = df_history[date_column].max()
        start_date = df_history[date_column].min()
    else:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

    # Calculate the date range
    date_range = end_date - start_date
    if date_range.days < 0:
        # print("Error: End date is earlier than start date.")
        return {}

    historical_periods = {}
    current_year = end_date.year  # Use the end_date year as the "current" year
    available_years = sorted(df[date_column].dt.year.unique(), reverse=True)

    # Iterate through available years *excluding* the "current" year
    for year in available_years:
        if year == current_year:
            continue  # skip current year

        # Calculate the start and end dates for the current year
        year_start_date = pd.to_datetime(f"{year}-{start_date.month:02}-{start_date.day:02}")
        year_end_date = pd.to_datetime(f"{year}-{end_date.month:02}-{end_date.day:02}")

        # Filter the DataFrame for the current year's date range
        year_df = df[(df[date_column] >= year_start_date) &
                             (df[date_column] <= year_end_date)].copy()

        if not year_df.empty:
            historical_periods[year] = year_df
        else:
            # print(f"No data found for {year}.")
            historical_periods[year] = pd.DataFrame()

    last_28days_dict = historical_periods
    counter = 0
    final_last_28_days = pd.DataFrame()

    for i in last_28days_dict.keys():
        df = last_28days_dict[i]
        if not df.empty and not df.isna().all().all():
            if counter == 0:
                final_last_28_days = pd.concat([final_last_28_days, df], ignore_index=True)
                final_last_28_days['Year'] = final_last_28_days['Date'].dt.year
                counter += 1
            else:
                break


    return historical_periods


@tool
def get_historical_periods_dynamic_fcst(input: str) -> str:
    """
    Forecast is Extracted for the same date range from forecast dataframe, dynamically determining available years.
    Returns:
        dict: A dictionary where keys are years and values are DataFrames
              containing the corresponding period. Returns an empty dictionary if there are issues.
    """
    # Ensure date column is datetime
    global df,df_history,historical_periods_fcst

    start_date = None
    end_date = None

    date_column='Date'
    df_history = df_history.reset_index()
    df_history[date_column] = pd.to_datetime(df_history[date_column])

    # Determine start and end dates if not provided
    if start_date is None or end_date is None:
        end_date = df_history[date_column].max()
        start_date = df_history[date_column].min()
    else:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

    # Calculate the date range
    date_range = end_date - start_date
    if date_range.days < 0:
        # print("Error: End date is earlier than start date.")
        return {}

    historical_periods = {}
    current_year = end_date.year  # Use the end_date year as the "current" year
    available_years = sorted(df[date_column].dt.year.unique(), reverse=True)

    # Iterate through available years *excluding* the "current" year
    for year in available_years:
        if year == current_year:
            continue  # skip current year

        # Calculate the start and end dates for the current year
        year_start_date = pd.to_datetime(f"{year}-{start_date.month:02}-{start_date.day:02}")
        year_end_date = pd.to_datetime(f"{year}-{end_date.month:02}-{end_date.day:02}")

        # Filter the DataFrame for the current year's date range
        year_df = df[(df[date_column] >= year_start_date) &
                             (df[date_column] <= year_end_date)].copy()

        if not year_df.empty:
            historical_periods[year] = year_df
        else:
            # print(f"No data found for {year}.")
            historical_periods[year] = pd.DataFrame()

    last_28days_dict = historical_periods
    counter = 0
    final_last_28_days = pd.DataFrame()

    for i in last_28days_dict.keys():
        df = last_28days_dict[i]
        if not df.empty and not df.isna().all().all():
            if counter == 0:
                final_last_28_days = pd.concat([final_last_28_days, df], ignore_index=True)
                final_last_28_days['Year'] = final_last_28_days['Date'].dt.year
                counter += 1
            else:
                break


    return historical_periods_fcst

tools = [get_historical_periods_dynamic_actual,get_historical_periods_dynamic_fcst]

llm = ChatVertexAI(
    model="gemini-2.5-pro",
    project="steady-bonsai-467007-g0",
    location="us-central1",
    max_output_tokens=1024,
    temperature=0,
    credentials=credentials
)
bound_llm = llm.bind_tools(tools)

agent_node = create_react_agent(bound_llm, tools=tools, state_schema=AgentState)
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)
graph.set_entry_point("agent")


runnable = graph.compile()
output = runnable.invoke({
    "messages": [{"role": "user", "content": "Extracts the same date range in previous years where using  'df' and 'df_history' where df is actual and df_history is last 28 days"}]
})

output_2 = runnable.invoke({
    "messages": [{"role": "user", "content": "Forecast is Extracted for the same date using  'forecast_df' and 'df_history_2' "}]
})

print("\nFinal output from agent:\n")
for msg in output["messages"]:
    if hasattr(msg, "name"):
        print(f"[TOOL: {msg.name}] {msg.content}")



Final output from agent:

[TOOL: None] Extracts the same date range in previous years where using  'df' and 'df_history' where df is actual and df_history is last 28 days
[TOOL: None] 
[TOOL: get_historical_periods_dynamic_actual] {np.int32(2025):           Date  Call Volume day_of_week  month  quarter  is_weekend  \
616 2025-08-01      37293.0      Friday      8        3           0   
624 2025-08-02      24235.0    Saturday      8        3           1   
632 2025-08-03      20968.0      Sunday      8        3           1   
703 2025-09-01      41307.0      Monday      9        3           0   
711 2025-09-02      20031.0     Tuesday      9        3           0   
719 2025-09-03      24262.0   Wednesday      9        3           0   
790 2025-10-01      37743.0   Wednesday     10        4           0   
798 2025-10-02      40694.0    Thursday     10        4           0   
806 2025-10-03      41303.0      Friday     10        4           0   
877 2025-11-01      23184.0    Saturday  

In [ ]:
#ROUGH